# EP 05 - format 인자와 다양한 Renderer

`rest_framework.response.Response`에서는 2가지 타입의 응답을 할 수 있습니다.

+ `api` : API Endpoint에 브라우저를 통해 접근할 때, 웹UI로 API를 조회할 수 있습니다.
+ `json` : 보통의 API 접근 시

API 요청에서는 `format=json`으로 디폴트 처리되며, 웹브라우저를 통해 요청될 때에는 `format=api`로 디폴트 처리됩니다.

APIView에서는 출력포맷을 결정하는 format인자를 다음 3가지 형태로 받을 수 있습니다.

+ Accept 헤더
+ GET인자 format
+ URL끝에 ".포맷"

지정 예) JSON 응답 요청

```
http :8000/ep04/ Accept:application/json
http :8000/ep04/?format=json
http :8000/ep04.json
```

지정 예) HTML 응답 요청

```
http :8000/ep04/ Accept:text/html
http :8000/ep04/?format=api
http :8000/ep04/.api
```

헤더와 GET인자를 활용하는 방법은 별도의 URL Conf 설정이 필요없습니다. 하지만 `:8000/.json`의 경우에는 "인자 Capture"가 필요하므로 별도의 URL Conf 설정이 필요하며 뷰에서도 `format` 이름의 인자를 받을 수 있어야 합니다.

URLConf에서 format인자에 대한 패턴을 직접 정의할 필요는 없습니다. 기존 URL Conf설정을 활용하여 format 인자 지원을 `rest_framework`에서 도와줍니다. 만약 다음 2가지 URL이 있다면

+ `/posts/`
+ `/posts/1/`

다음 URL도 사용토록 도와줍니다.

+ `/posts.json` : 포스팅 목록 (JSON 포맷)
+ `/posts/1.json` : 1번 포스팅 (JSON 포맷)
+ `/posts.api` : 포스팅 목록 (HTML 포맷)
+ `/posts/1.api` : 1번 포스팅 (HTML 포맷)

실제 코드를 살펴봅시다. 

```python
# myapp/urls.py

from rest_framework.urlpatterns import format_suffix_patterns

urlpatterns = [
    # 이미 여러 url pattern이 등록되어있을 것입니다.
]

# 기존 urlpatterns 끝에 추가합니다. 이를 통해 인자로 넘겨진 urlpatterns에 suffix지원을 추가해줍니다.
urlpatterns = format_suffix_patterns(urlpatterns)
```

하지만, `DefaultRouter`를 사용하고 계시다면, 위 `format_suffix_patterns`처리를 추가로 하실 필요가 없습니다. `DefaultRouter`내에서 이미 처리되어있습니다.

`ViewSet`에서는 별도로 신경써줄 것은 없으며, `@api_view`장식자를 적용한 함수 뷰에서는 `format인자`를 받을 수 있도록 `Keyword 인자`를 필히 지정해줘야합니다. 인자만 받을 수 있도록 할 뿐 추가로 구현해야할 것은 없습니다. `api_view`와 `Response`를 통해 `format`인자가 처리됩니다.

```python
# myapp/views.py

from rest_framework.decorators import api_view

@api_view(['GET', 'POST'])
def post_list(request, format=None):
    # Response 활용

@api_view(['GET', 'PUT', 'DELETE'])
def post_detail(request, pk, format=None):
    # Response 활용
```

이제, 다음 URL을 통해 format인자를 지정해보세요.

+ `http :8000/ Accept:application/json`
+ `http :8000/?format=json`
+ `http :8000/.json`

+ `http :8000/ Accept:text/html`
+ `http :8000/?format=api`
+ `http :8000/.api`

# 기본 지원되는 Renderer 몇 가지

+ **`JSONRenderer`** (디폴트 지정) : `json.dumps`를 통한 JSON 직렬화
    - `format`: `json`
+ `BrowsableAPIRenderer` (디폴트 지정) : self-document HTML 렌더링
    - `format`: `api`

+ `TemplateHTMLRenderer` : 지정 템플릿을 통한 렌더링
    - `format`: `html`
    - `template_name`: 지정 필요
    - 템플릿을 찾는 순서
        - Response에서 `template_name` 찾기 : `return Response({}, template_name='mytempalte.html')`
        - Renderer에서 `template_name` 찾기
        - 뷰 클래스에서 `get_template_names` 함수 찾기
        - 뷰 클래스에서 `template_name` 속성 찾기


## API를 사용하는 HTML 페이지 만들기

참고: http://www.django-rest-framework.org/topics/html-and-forms/

Django Form을 사용하지 않고, DRF를 통해 HTML페이지에서; 폼처리를 할 수는 있습니다. 하지만 이 부분은 DRF를 쓰기보다 Django Form을 사용하는 것이 보다 효율적인 듯 해서, 다루진 않겠습니다.

언젠가 Django Form과 Serializer가 합쳐지는 날이 오지 않을까 싶네요.

---

Life is short. Use Python3/Django.

\- AskDjango